#### The Two Wise Men Problem

Two wise men, Sage P and Sage S, are given two different natural numbers 𝑥 and 𝑦, where 1<𝑥,𝑦<100.

Sage P is told the product 𝑃=𝑥⋅𝑦, while Sage S is told the sum 𝑆=𝑥+𝑦. Neither of them knows the actual values of 𝑥 and 𝑦.

The following conversation takes place:

Sage P: "I don't know what the numbers are."

Sage S: "I knew that already."

Sage P: "Oh, now I know the numbers!"

Sage S: "And now I do too!"

What are the numbers 𝑥 and 𝑦?

In [2]:
limit = 100

# Створюємо словник для зберігання кількості входжень кожного добутку x * y до їх розмови.
allowable_products_before = {} 
for x in range(2, limit): 
    for y in range(x + 1, limit - x): 
        product = x * y
        # Якщо цей добуток ще не був зустрінутий, додаємо його до словника.
        if product not in allowable_products_before: 
            allowable_products_before[product] = 1 
        # Якщо цей добуток вже був, збільшуємо кількість його входжень.
        else:
            allowable_products_before[product] += 1

# Створюємо словник для зберігання заборонених сум x + y, коли P говорить "Я не знаю".
not_allowed_say_p = {}  
for x in range(2, limit): 
    for y in range(x + 1, limit - x): 
        product = x * y
        # Якщо кількість входжень цього добутку дорівнює 1, то сума x + y недопустима.
        if allowable_products_before[product] == 1 :
            not_allowed_say_p[x + y] = 1  

# Створюємо словник для зберігання кількості входжень кожного добутку x * (n - x),
# де n є сумою x і y, яка не була відмічена як недопустима.
allowable_products_after_p = {} 
for n in range(2, limit):
    if n not in not_allowed_say_p:
       for x in range(2, n // 2 + 1):
            product = x * (n - x)
            # Якщо цей добуток входить до списку розмови та має більше одного входження,
            # додаємо його до словника.
            if product in allowable_products_before and allowable_products_before[product] > 1:
               if product in allowable_products_after_p:
                   allowable_products_after_p[product] += 1
               else:
                   allowable_products_after_p[product] = 1 

# Створюємо словник для зберігання кількості входжень суми x + y,
# яка може бути поділена на x і (n - x), коли S говорить "Я не знаю".
allowable_say_s = {}  
for n in range(2, limit):
    if n not in not_allowed_say_p:
       for x in range(2, n // 2 + 1):
           product = x * (n - x)
           # Якщо цей добуток входить до списку розмови та має тільки одне входження,
           # додаємо його до словника.
           if product in allowable_products_after_p and allowable_products_after_p[product] == 1:
               if n in allowable_say_s:
                   allowable_say_s[n] += 1
               else:
                   allowable_say_s[n] = 1

# Виводимо результати, де кількість входжень суми n дорівнює 1.
for n in allowable_say_s: 
    if allowable_say_s[n] == 1:
        for x in range(2, n // 2 + 1):
            product = x * (n - x)
            # Якщо цей добуток входить до списку розмови та має тільки одне входження,
            # виводимо його разом з значеннями x та (n - x).
            if product in allowable_products_after_p and allowable_products_after_p[product] == 1:
                print('(Сума, Добуток) = (%d, %d), (x, y) = (%d, %d)' % (n, product, x, n - x))

(Сума, Добуток) = (17, 52), (x, y) = (4, 13)


In [3]:
import numpy as np

# Build the 2d array of pairs of numbers in the interval [2, 99]
# Note: the order of numbers in the pair is not important, i.e. the pair (2, 3) is the same as the pair (3, 2)
numbers = np.array([
    [a, b]
    for a in range(2, 100)
    for b in range(a, 100)
])

# To reduce the range of possible pairs of numbers, "Prod" can consider only
# integers in the interval [2, 99] whose product is equal to given him number
# ---------------------------------------------------------------------------
# Each key of the dictionary is related to the 2d array
# of pairs of numbers whose product is equal to that key
prods = {
    p: numbers[numbers.prod(axis=1) == p]
    for p in np.unique(numbers.prod(axis=1))
}

# "Sum" can do the same with given him number
# -----------------------------------------------------
# Each key of the dictionary is related to the 2d array
# of pairs of numbers whose sum is equal to that key
sums = {
    s: numbers[numbers.sum(axis=1) == s]
    for s in np.unique(numbers.sum(axis=1))
}

print("In total, there are:")
print(f"- {len(numbers)} pairs of numbers in interval [2, 99]")
print(f"- {len(prods)} unique products of those pairs of numbers")
print(f"- {len(sums)} unique sums of those pairs of numbers")

In total, there are:
- 4851 pairs of numbers in interval [2, 99]
- 2843 unique products of those pairs of numbers
- 195 unique sums of those pairs of numbers


In [4]:
# Update the pairs of numbers:
# consider the pairs related with each sum, and keep only those whose products have more than one related pair
numbers = np.concatenate([
    pairs
    for pairs in sums.values()
    if all(prods[p].shape[0] != 1 for p in pairs.prod(axis=1))
])

print(f"Count of updated pairs of numbers: {len(numbers)}")

Count of updated pairs of numbers: 145


In [5]:
# Define new pairs for products based on updated pairs of numbers
prods = {
    p: numbers[numbers.prod(axis=1) == p]
    for p in np.unique(numbers.prod(axis=1))
}

# Update the pairs of numbers:
# consider the pairs related with each product, and keep only those that are single for a related product
numbers = np.concatenate([
    pairs
    for pairs in prods.values()
    if pairs.shape[0] == 1
])

print(f"Count of updated pairs of numbers: {len(numbers)}")

Count of updated pairs of numbers: 86


In [6]:
# Define new pairs for sums based on updated pairs of numbers
sums = {
    s: numbers[numbers.sum(axis=1) == s]
    for s in np.unique(numbers.sum(axis=1))
}

# Update the pairs of numbers:
# consider the pairs related with each sum, and keep only those that are single for a related sum
numbers = np.concatenate([
    pairs
    for pairs in sums.values()
    if pairs.shape[0] == 1
])

print(f"Count of updated pairs of numbers: {len(numbers)}")

Count of updated pairs of numbers: 1


In [7]:
# Print the numbers that are left
a, b = np.squeeze(numbers)
print(f"Determinated numbers are {a} and {b}")

Determinated numbers are 4 and 13
